In [1]:
import pandas as pd

In [4]:
path = 'DATA/tariff_2022.xlsx'

grid_payment = pd.read_excel(
    path,
    skiprows=1,
    decimal=','
).rename(columns={'Unnamed: 0' : 'regions'})

grid_payment = grid_payment.set_index('regions')
grid_payment.index = grid_payment.index.str.rstrip()

# # #

sales_payment = pd.read_excel(
    path,
    sheet_name=1,
    decimal=',',
    names=['Регион', 'ГП', '<.670', '>.670', '>10']
)

sales_payment = sales_payment.set_index('Регион')
sales_payment.index = sales_payment.index.str.rstrip()

# # #

prices = pd.read_excel(
    path,
    sheet_name=2,
    index_col=0,
    parse_dates=True
)

# # #

power = pd.read_excel(
    path,
    sheet_name=3,
    index_col=0,
    parse_dates=True
)

# # #

peaks = pd.read_excel(
    path,
    sheet_name=5,
    index_col=0,
    parse_dates=True
)

# # #

working_days = pd.DataFrame(index=prices.index)
working_days['year'] = working_days.index.year
working_days['month'] = working_days.index.month
working_days['dummy'] = 0

working_days = working_days.groupby(['year', 'month']).count().loc[2022]

In [18]:
def cost_calc_3(region, voltage_class='ВН', max_power='>10', K_energy=1, K_power=1):
    """
    Расчет стоимости электроэнергии по 3-й ЦК
    
    Parameters
    -----------
        region - str
            имя региона
        voltage_class - str
            класс напряжения (ВН, СН II, СН I, НН)
        max_power - str
            максимальная мощность эн.прин. энергоуст. потребителя (<.670, >.670, >10)
        K_energy - float
            множитель тарифной ставки на ЭЭ
        K_power - float
            множитель тарифной ставки на "генераторную" мощность
    
    Return
    -----------
        
    
    """
    
    peak_hours = peaks[[region]]
    peak_hours = peak_hours.set_index(region)
    peak_hours.index.name = 'timestamp'
    peak_hours['peak'] = 1

    df = prices[[region]].join(peak_hours).fillna(0)
    df = df.rename(columns={region : 'price'})
    df['peak'] = df['peak'].astype(int)

    df = df.join(power[[region]]).fillna(method='ffill')
    df = df.rename(columns={region : 'power'})

    transport = grid_payment.loc[region, voltage_class]
    sales = sales_payment.loc[region, max_power] * 10 

    try:
        df['price'] += transport + float(sales)
    except:
        df['price'] += transport + float(sales[0])
    df['price'] += 1.681 + 2.182 + 0.393
    
    df = df.join(load)

    # # #

    df['price'] = df['price'] * K_energy
    df['power'] = df['power'] * K_power

    # # #

    df['year'] = df.index.year
    df['month'] = df.index.month

    df['energy_cost'] = df['price'] * df['load']
    df['power_cost']  = df['peak'] * df['power'] * df['load']

    energy_cost = df.groupby(['month', 'year'])[['energy_cost']].sum()
    power_cost = df[df['power_cost'] > 0].groupby(['month', 'year'])[['power_cost']].sum()
    power_cost = power_cost.join(working_days)
    power_cost['power_cost_mean'] = power_cost['power_cost'] / power_cost['dummy']

    total_cost = energy_cost['energy_cost'].sum() + power_cost['power_cost_mean'].sum()

    return total_cost, df

In [22]:
load = pd.DataFrame(index=pd.date_range(start='2022-01-01 00:00:00', end='2022-12-31 23:00:00', freq='1H'))
load['load'] = 1
load

,load
2022-01-01 00:00:00,1
2022-01-01 01:00:00,1
2022-01-01 02:00:00,1
2022-01-01 03:00:00,1
2022-01-01 04:00:00,1
...,...
2022-12-31 19:00:00,1
2022-12-31 20:00:00,1
2022-12-31 21:00:00,1
2022-12-31 22:00:00,1


In [23]:
total_cost, df = cost_calc_3('Кемеровская область', 'НН', '<.670')
total_cost

48840765.04003901